<a href="https://colab.research.google.com/github/inhopp/ML_code/blob/main/MoveNet_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

## Load Model

In [ ]:
!mkdir ./model
!wget -O ./model/MoveNet_lightning https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite

--2022-02-21 18:45:17--  https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
Resolving tfhub.dev (tfhub.dev)... 108.177.111.138, 108.177.111.139, 108.177.111.101, ...
Connecting to tfhub.dev (tfhub.dev)|108.177.111.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-lite-models/google/lite-model/movenet/singlepose/lightning/tflite/float16/4.tflite [following]
--2022-02-21 18:45:18--  https://storage.googleapis.com/tfhub-lite-models/google/lite-model/movenet/singlepose/lightning/tflite/float16/4.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.128, 173.194.196.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4758512 (4.5M) [application/octet-stream]
Saving to: ‘./model/MoveNet_lightning’

./model/MoveNet_lig 100%[=

In [ ]:
interpreter = tf.lite.Interpreter(model_path='./model/MoveNet_lightning')
interpreter.allocate_tensors()

## Video Record

In [ ]:
from IPython.display import display, Javascript,HTML
from google.colab.output import eval_js
from base64 import b64decode
from base64 import b64encode

In [ ]:
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
    
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);

      video.srcObject = stream;
      video.muted = true;

      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);

      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))

In [ ]:
def show_video(video_path, video_width = 600):
  
  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
video_path = "test.mp4"
record_video(video_path)

<IPython.core.display.Javascript object>

Finished recording video at:test.mp4


In [ ]:
show_video(video_path)

## Make Detections

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
  y, x, c = frame.shape
  shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

  for kp in shaped:
    ky, kx, kp_conf = kp

    if kp_conf > confidence_threshold:
      cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) # 위치, 크기, 색, 두꼐 -1:색 채워넣기

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [ ]:
import cv2
video_input_path = 'test.mp4'
video_output_path = 'test_output.mp4'

In [ ]:
cap = cv2.VideoCapture(video_input_path)

codec = cv2.VideoWriter_fourcc(*'XVID')

vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
vid_fps = cap.get(cv2.CAP_PROP_FPS)

vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size)

while True:
  ret, frame = cap.read()

  if not ret:
    break

  # Reshape image
  img = frame.copy()
  img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
  input_image = tf.cast(img, dtype=tf.uint8)

  # Setup input and output
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  # Make predictions
  interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
  interpreter.invoke()
  keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

  # Rendering
  draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
  draw_keypoints(frame, keypoints_with_scores, 0.4)

  vid_writer.write(frame)


vid_writer.release()
cap.release()





In [ ]:
show_video(video_output_path)